# Vectorize using neighbor information?

Use thisnotthat kernel.
conda env update --file TNT/environment.yml --prune

Did not pursue because HDBSCAN does not handle reading a sparse matrix as entry.

In [1]:
!git branch

* main


In [2]:
execfile('functions/data_specifics.py')
execfile('functions/graph_functions.py')
print(data_set_list)

['pendigits', 'coil', 'mnist', 'usps', 'buildings', 'clusterable']


In [3]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
from IPython.display import display, Markdown, Latex

import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns

import hdbscan
import umap
from sklearn.neighbors import KNeighborsTransformer
import pynndescent
sns.set()

In [8]:
import vectorizers as vz
from vectorizers import transformers as vzt

In [6]:
def eval_clusters(cluster_labels0, true_labels, raw_data, cluster_method="None", min_cluster_size=1):
    cluster_labels = cluster_labels0.copy()
    unique_labels = np.unique(cluster_labels)
    cluster_sizes, size_ids = np.histogram(cluster_labels, bins=unique_labels)
    if np.any(cluster_sizes <= min_cluster_size): # Has singleton clusters -- call them noise
        singleton_clusters = size_ids[:-1][cluster_sizes < min_cluster_size]
        for c in singleton_clusters:
            cluster_labels[cluster_labels == c] = -1
    if (np.any(cluster_labels < 0)): # Has noise points
        clustered_points = (cluster_labels >= 0)
        ari = adjusted_rand_score(true_labels[clustered_points], cluster_labels[clustered_points])
        ami = adjusted_mutual_info_score(true_labels[clustered_points], cluster_labels[clustered_points])
        sil = silhouette_score(raw_data[clustered_points], cluster_labels[clustered_points])
        pct_clustered = (np.sum(clustered_points) / cluster_labels.shape[0])
        #print(f"ARI: {ari:.4f}\nAMI: {ami:.4f}\nSilhouette: {sil:.4f}\nPct clustered: {pct_clustered * 100:.2f}%")
    else:
        ari = adjusted_rand_score(true_labels, cluster_labels)
        ami = adjusted_mutual_info_score(true_labels, cluster_labels)
        sil = silhouette_score(raw_data, cluster_labels)
        #print(f"ARI: {ari:.4f}\nAMI: {ami:.4f}\nSilhouette: {sil:.4f}")
        pct_clustered = 1.0
    
    return {"Method": cluster_method, "ARI": ari, "AMI": ami, "Silhouette": sil, "Pct Clustered": pct_clustered}

# Pendigits clustering scores

In [9]:
dataset_id = 0
raw_data, targets, dataset_name = get_dataset(dataset_id)

A, sigmas, rhos, dists = get_umap_graph(raw_data, dataset_id=dataset_id, set_op_mix_ratio=1.0, return_only_matrix=True)

In [10]:
A0 = 1*(A>0)

In [11]:
import thisnotthat as tnt
import panel as pn
from sklearn.preprocessing import RobustScaler
import seaborn as sns
import umap

In [12]:
clusterer = hdbscan.HDBSCAN().fit_predict(A0.toarray())
clusterer

array([-1, -1, 23, ..., -1, 17, -1])

In [13]:
eval_clusters(clusterer, targets, raw_data, cluster_method="None", min_cluster_size=1)

{'Method': 'None',
 'ARI': 0.5962509239278841,
 'AMI': 0.763499452387987,
 'Silhouette': 0.19609363,
 'Pct Clustered': 0.49081803005008345}

In [14]:
A_info = vzt.InformationWeightTransformer().fit_transform(A0)

In [15]:
clusterer_info = hdbscan.HDBSCAN().fit_predict(A_info.toarray())
clusterer_info

array([-1, -1, 18, ..., -1, 24, -1])

In [16]:
eval_clusters(clusterer_info, targets, raw_data, cluster_method="None", min_cluster_size=1)

{'Method': 'None',
 'ARI': 0.627210091250614,
 'AMI': 0.7880989790097518,
 'Silhouette': 0.19868945,
 'Pct Clustered': 0.5420144685587089}

In [17]:
def cluster_using_neighbors(dataset_id):
    raw_data, targets, dataset_name = get_dataset(dataset_id)
    A, sigmas, rhos, dists = get_umap_graph(raw_data, dataset_id=dataset_id, set_op_mix_ratio=1.0, return_only_matrix=True)
    A0 = 1*(A>0)
    A_info = vzt.InformationWeightTransformer().fit_transform(A0)
    clusterer_info = hdbscan.HDBSCAN().fit_predict(A_info.toarray())
    return(eval_clusters(clusterer_info, targets, raw_data, cluster_method=None, min_cluster_size=1))

In [ ]:
for i in range(len(data_set_list)):
    res = cluster_using_neighbors(dataset_id=i)
    res['Data set'] = data_set_list[i]

/disk/home/vmpouli/.conda/envs/tnt_env/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
res